In [1]:
!pip install langchain pinecone-client google-generativeai openai tqdm chromadb langchain_community langchain-google-genai pypdf PyPDF2 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0

In [6]:
!pip install CSVLoader

  Preparing metadata (setup.py) ... done
  Created wheel for CSVLoader: filename=csvloader-0.0.2-cp310-cp310-linux_x86_64.whl size=14067 sha256=5d95fa9d49af9a8ea0aa61b4a0bf403c462aa2f143000c38d2016df55707eabc
  Stored in directory: /root/.cache/pip/wheels/82/40/cb/0a5d24f0e3d3520d4db12b236d05712e819671dce42cee42bc
Successfully built CSVLoader


In [2]:

from google.colab import userdata
key=userdata.get('GOOGLE_API_KEY')


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key=key,
                                   model="gemini-1.5-flash")

# RAG

In [8]:





# import csv loader
from langchain.document_loaders.csv_loader import CSVLoader
# import text splitter
from langchain.text_splitter import CharacterTextSplitter

# Load the PDF using PyPDFLoader
loader = CSVLoader("/content/faq_bot_university.csv")
documents = loader.load_and_split()  # Returns a list of Document objects

In [9]:
# to show some lines of documents
documents[0].page_content[:100]

'Question: What is the tuition fee for undergraduate programs?\nAnswer: The tuition fee for undergradu'

In [10]:

# Extract the text content from the Document objects
pages = [doc.page_content for doc in documents]

In [11]:
len(pages)

1000

In [12]:
# Initialize the CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Split the text into chunks
chunks = text_splitter.create_documents(pages)

# Display results
print(f"Number of chunks: {len(chunks)}")
print(f"Type of first chunk: {type(chunks[0])}")

Number of chunks: 1000
Type of first chunk: <class 'langchain_core.documents.base.Document'>


In [13]:
chunks[3]

Document(metadata={}, page_content='Question: Can international students apply for financial aid?\nAnswer: Yes, international students can apply for financial aid.')

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=key, model="models/embedding-001")

In [15]:
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=key,  model="models/text-embedding-004",  chunk_size=100,task_type="retrieval_document")


In [16]:

from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(documents, embedding=embedding_model)

In [17]:
llm = ChatGoogleGenerativeAI(google_api_key=key,
                                   model="gemini-1.5-flash")

In [18]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [19]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [20]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot.
                  Given a context and question from user,
                  you should answer based on the given context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context: {context}
    Question: {question}
    Answer: """)
])

In [21]:

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [22]:

from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [23]:
response = rag_chain.invoke("""Please summarize Leave No Context Behind:
                            Efficient Infinite Context Transformers with Infini-attention""")

response

'I am sorry, but I do not have access to external websites or specific files online, including research papers.  Therefore, I cannot summarize the paper "Leave No Context Behind: Efficient Infinite Context Transformers with Infini-attention".  To get a summary, you will need to access and read the paper yourself.\n'

In [24]:
response = retriever.invoke("""Please summarize Leave No Context Behind:
                            Efficient Infinite Context Transformers with Infini-attention""")

response

[Document(metadata={'row': 292, 'source': '/content/faq_bot_university.csv'}, page_content='Question: What courses are offered in Computer Science?\nAnswer: Courses include AI, Data Science, and Web Development.'),
 Document(metadata={'row': 607, 'source': '/content/faq_bot_university.csv'}, page_content='Question: What courses are offered in Computer Science?\nAnswer: Courses include AI, Data Science, and Web Development.'),
 Document(metadata={'row': 457, 'source': '/content/faq_bot_university.csv'}, page_content='Question: What courses are offered in Computer Science?\nAnswer: Courses include AI, Data Science, and Web Development.'),
 Document(metadata={'row': 342, 'source': '/content/faq_bot_university.csv'}, page_content='Question: What courses are offered in Computer Science?\nAnswer: Courses include AI, Data Science, and Web Development.'),
 Document(metadata={'row': 747, 'source': '/content/faq_bot_university.csv'}, page_content='Question: What courses are offered in Computer S